# Aula 2 - Separabilidade Não Linear e *Support Vector Machines*

Na aula de hoje vamos continuar o tema iniciado na aula passada, e vamos acrescentar algumas complexidades ao problema:

- 1. Separabilidade Não-Linear
- 2. Feature Maps
- 3. Funções de Kernel
- 4. SVM não lineares
- 5. SVM para Regressão

## 1. Separabilidade Não Linear

Recapitulando o que falamos na aula passada:

> Um conjunto de dados é dito **linearmente separável** se existe algum hiperplano que divide perfeitamente os registros entre as categorias existentes. Nesse caso, o hiperplano é conhecido como *classificador de margem*.

Vimos também que o classificador de margem máxima é aquele com maior potencial de generalização, mas que no caso de conjuntos de dados muito ruidosos ou com valores extremos, pode ser necessário um grau de folga na margem, sendo característico do *classificador de margem suave*, também conhecido como *Support Vector Machine* ou SVM. O classificador de margem suave introduz um pouco mais de viés, porém com grande potencial de diminuir a variância do modelo.

Em todos os casos (margem fixa ou margem suave), a fronteira de decisão será sempre linear no espaço dos atributos.

No entanto, vamos considerar o seguinte caso de conjunto de dados unidimensional

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/e00c77e0-14fb-472f-b9ae-f6af7b229ad5.PNG width=400>

É evidente que este dataset **não é linearmente separável**! Portanto, não conseguimos produzir um classificador de vetores de suporte para este dataset **no espaço de features original**.

## 2. Feature Maps

Mas, aí entra uma ideia muito interessante: e se nós **levarmos os dados para um ou outro espaço?**

Seria possível que no espaço original os dados não sejam linearmente separáveis, mas **o sejam** em algum outro espaço?

Bom, a priori, vamos tentar algo bem simples... Que tal introduzirmos **uma nova feature** $X_2 = X_1^2$? O que aconteceria neste caso?

De fato, ao **introduzirmos** uma nova feature, estamos fazendo com que **cada observação passe a ser caracterizada por duas features ao invés de uma única**!

Ou seja, nosso espaço de features efetivamente muda de $\mathbb{R}^1$ para $\mathbb{R}^2$! Veja:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/8d5e6199-8a33-45cf-9e4f-c6507024fb36.PNG width=800>

O procedimento que fizemos é chamado de **feature map**, e ele é matematicamente representado pelo mapa (função) $\Phi$. 

> Como $\Phi$ leva observações do espaço original ( $1$ D, uma única feature $X_1$, para vetores do novo espaço ($2$ D, duas features, $X_1$ e $X_2$), o denotamos como: 
$$\begin{align*}
\Phi \ \colon \ & \mathbb{R}^1 \longrightarrow \mathbb{R}^2 \\
& \vec{x} = X_1 \longmapsto \Phi(\vec{x}) = (X_1, X_2)
\end{align*}$$

Um pouco mais de terminologia:

> O "espaço original" é comumente chamado de **espaço de input** (representaremos por $\mathcal{X}$); enquanto o espaço após a aplicação do feature map é chamado de **espaço de features (representaremos por $\mathcal{Z}$)**

<img src=https://miro.medium.com/max/872/1*zWzeMGyCc7KvGD9X8lwlnQ.png width=400>

Também é comum se referir ao espaço de features como **espaço z**, devido à comum notação $\Phi(\vec{x}) \equiv \vec{z}$. Neste caso, teríamos:

$$\begin{align*}
\Phi \ \colon \ & \mathbb{R}^1 \longrightarrow \mathbb{R}^2 \\
& \vec{x} = X_1 \longmapsto \Phi(\vec{x}) = \vec{z} = (Z_1, Z_2)
\end{align*}$$

Uma vez que temos estas definições, podemos perceber a real utilidade do feature map: **os dados não eram linearmente separáveis no espaço de input, mas passaram a ser no espaço de features!**

Isso é realmente formidável, pois, se temos dados linearmente separáveis, podemos **treinar um classificador de margem suave** no espaço de features!

Isso pode parecer estranho, pois, afinal, gostaríamos de separar os dados no espaço original, não é mesmo?

Na verdade, nosso objetivo é que os dados sejam separados, **não importa em que espaço**! Se conseguirmos encontrar um espaço onde há separabilidade através da aplicação de um feature map, bastaria **aplicar o mesmo feature map** aos dados de treino e de teste, e a separabilidade sempre estará garantida!

Vejamos mais um exemplo, agora projetando um 2D para 3D

$$\phi(x_1, x_2) = (z_1, z_2, z_3) = (x_1, x_2, e^{-|x_1^2 + x_2^2|})$$

> Uma **Support Vector Machine** nada mais é que **um classificador de margem suave** treinado **no espaço de features**. Portanto, este classificador pressupõe a aplicação prévia de um **feature map** aos dados no espaço de input.

## 3. Funções de Kernel

Da discussão acima, ficou claro que é justamente o feature map que dá grande poder às SVMs. De fato, a possibilidade de conseguirmos separabilidade linear é algo formidável!

Neste contexto, uma pergunta natural é: como escolher um bom feature map? Formalmente, existem infinitos feature maps possíveis! Como escolher, dentre infinitas opções, exatamente o mapa exato que nos garante separabilidade linear no espaço de features? Embora esta pergunta não seja fácil de responder, existem algumas técnicas para nos ajudar a escolher bons feature maps (vamos discutir sobre isso mais a diante).

Além disso, existe um segundo problema, ainda maior: suponha que queiramos introduzir um kernel que leva os pontos para um espaço de features de altíssima dimensionalidade (algo como $\Phi : \mathbb{R}^2 \mapsto \mathbb{R}^{10000}$).

É de se esperar que este seja um feature map **operacionalmente custoso** de ser calculado, não é mesmo? Imagina, ter que aplicar esta transformação para todos os pontos de treino, e depois de teste! De fato, isso rapidamente se torna computacionalmente impraticável...

Pra solucionar este problema, foi introduzido o uso de **funções de kernel** para capturar um aspecto importante dos feature maps!

Mas, pra entendermos a importância das funções de kernel, é necessário entendermos uma coisa:

> A hipótese do SVM depende apenas do **produto interno** entre as observações no espaço de features!!

Uma função de kernel $\kappa$ nada mais é que uma **medida de similaridade** entre dois vetores $\vec{x}$ e $\vec{x}'$ (que no nosso caso, são observações). Definimos como:

$$\boxed{\begin{align*}
\kappa \ \colon \ & \mathcal{X} \times \mathcal{X} \longrightarrow \mathbb{R} \\
& (\vec{x}, \vec{x}') \longmapsto \kappa(\vec{x}, \vec{x}')
\end{align*}}$$

Ou seja, um kernel é uma função que, dadas duas observações $\vec{x}$ e $\vec{x}'$, retorna um número real que caracteriza o quão similar as duas observações são entre si.

**Ponto fundamental**: uma função de kernel permite que **o produto escalar** entre duas observações seja calculado **no espaço de features**, sem que precisemos **explicitamente levar as observações pro espaço de features**.

Ou seja, conseguimos **evitar** que o feautre map, que costuma ser custoso computacionalmente, seja explicitamente aplicado!

E uma vez que pro SVM apenas o produto interno interessa, podemos usar diretamente a função de kernel, que é muito mais computacionalmente simples que o feature map explícito!

Esse é o chamado **kernel trick**.

### Tipos de Kernel Functions

Da mesma forma que existem infinitos feature maps possíveis, a variedade de kernels também é imensa! Apesar dos kernels oferecerem uma vantagem operacional absurda com relação à aplicação explícita do feature map, o problema de escolha de um kernel adequado para um determinado problema ainda existe.

Na prática, existem algumas formas de propor kernels, mas este não é um tema fácil. Existe todo um conjunto de métodos e técnicas que se utilizam de kerneles para tarefas de aprendizagem, os chamados [métodos de kernel](https://en.wikipedia.org/wiki/Kernel_method).

Apesar da enorme liberdade no design de kernels, há algumas classes particulares de kernels que são comumente utilizados:

- Linear kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \langle \vec{x}, \vec{x}'\rangle $
<br><br>
- Polynomial kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = (\gamma \langle \vec{x}, \vec{x}'\rangle + r)^d$
<br><br>
- Radial Basis Function (RBF) kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \exp(-\gamma \|\vec{x}-\vec{x}'\|^2)$
<br><br>
- Sigmoid kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \tanh(\gamma \langle \vec{x},\vec{x}'\rangle + r)$

> No exemplo explícito que fizemos acima, usamos justamente um kernel polinomial com $r=\gamma=1$ e $d=2$!

Note que a dependência funcional dos kernels muda, dependendo exatamente do feature map específico que eles representam. No entanto, em todos os casos, as features no espaço de input são utilizadas, o que garante a eficiência!

## 4. SVM não lineares

Para o desenvolvimento, vamos usar a classe `SVC` do pacote `scikit-learn`, cuja documentação está [nesse link](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

Para a construção de um modelo SVM, é muinto importante que os dados sejam normalizados!

O motivo é bem simples: como vimos acima, o SVM é completamente baseado no kernel, que por sua vez é dado por um produto interno. Já o produto interno, é altamente dependente da **escala das features** (lembre-se, $\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle = \left | \Phi(\vec{x}) \right | \left | \Phi(\vec{x}') \right | \cos \left ({\theta_{\vec{x}, \vec{x}'}} \right )$, isto é, a norma dos vetores influencia o produto interno!).

Portanto, para evitar que efeitos de escala influenciem a classificação, a normalização é um passo extremamente importante!

### Aplicando Validação cruzada para selecionar o Kernel e os parâmetros

#### Exercício 1

Aplique o conceito de SVM não linear para classificação do conjunto de dados de qualidade dos vinhos. No entanto, faça a seguinte transformação na variável `quality`:

- **Vinhos com notas 3 e 4**: vinho "ruim" - classe 0
- **Vinhos com notas 5 e 6**: vinho "comum" - classe 1
- **Vinhos com notas 7 e 9**: vinho "bom" - classe 2

## 5. SVM para Regressão

Embora fizemos nossa apresentação do SVM como um classificador, também é possível utilizar este método para regressão!

Todos os elementos do classificador SVM (margem, kernel, etc.) também são relevantes aqui.

A ideia é bem simples: utilização de um kernel para que um modelo de **regressão linear seja treinado no espaço de features**. No espaço de inputs, este modelo é refletido como uma regressão não-linear (da mesma forma que, no caso de classificação, fronteiras de decisão lineares no espaço de features são refletidas como fronteiras não-lineares no espaçõ de input).

A principal diferença é que o conceito de margem também está presente, de modo que **apenas alguns pontos efetivamente vão contriuir para a regressão**. Neste caso, são os pontos **dentro da margem** (região conhecida como $\epsilon-$ tubo) que serão estes vetores de suporte. Ou seja, pontos que estão fora da margem não contribuem para a função de custo.

<img src=https://www.saedsayad.com/images/SVR_5.png width=600>

Uma comparação entre classificadores e regressores SVM:

<img src=https://miro.medium.com/max/1100/1*XE9jt0r1yAW8LnliQ3mllQ.png width=600>

#### Exercício 2

Sabendo que a classe do `scikit-learn` que implementa a regressão com SVM é a `SVR`, implemente um modelo de regressão para o dataset Steel_industry_data e faça as seguintes análises:

- Elimine as seguintes colunas: `Date`, `WeekStatus` e `Day_of_week`
- Sua variável-alvo é a coluna: `Usage_kWh`
- Checar dados nulos. Se houver, eliminá-los
- Separe 30 % dos dados para teste
- Use o Random Search para selecionar os hiperparâmetros (**Cuidado** com o número de hiperparâmetros pois o conjunto de dados é grande)
- Use um objeto `KFold` para validação cruzada (use 5 folds)
- As métricas de seleção podem ser o $R^2$ ou o MAE
- Utilize o modelo do `Pipeline`
- Não se esqueça de aplicar o escalonamento dos dados (`StandardScaler`)